In [ ]:
import ipywidgets as widgets
from IPython.display import display
from stmpy import Machine, Driver
import random


class StudyHall:
    def load_images(self):
        self.red_on = open("images/red_on.png", "rb").read()
        self.red_off = open("images/red_off.png", "rb").read()
        self.green_on = open("images/green_on.png", "rb").read()
        self.green_off = open("images/green_off.png", "rb").read()

    def on_button_new_card(self, b):
        self.stm.send_signal('newCard')

    def display(self):
        self.button_new_card = widgets.Button(description="Swipe new card")
        self.button_new_card.on_click(self.on_button_new_card)

        self.red = widgets.Image(value=self.red_off, format='png', width=50, height=50)
        self.green = widgets.Image(value=self.green_off, format='png', width=50, height=50)

        self.info = widgets.Label(value="Waiting...")
        self.room_state = widgets.IntProgress(
            value=len(self.id_array),
            min=0,
            max=10,
            step=1,
            description='People:',
            bar_style='info',
            orientation='horizontal'
        )

        #self.list_of_ids = [str(i) for i in self.id_array]
        #widgets.HBox(self.list_of_ids)

        display(self.red, self.green, self.button_new_card, self.room_state, self.info)  #, self.list_of_ids)

    def __init__(self):
        self.id_array = []

        self.load_images()
        self.display()

    #IMAGES:

    def turn_red_on(self):
        self.red.set_trait(name='value', value=self.red_on)

    def turn_red_off(self):
        self.red.set_trait(name='value', value=self.red_off)

    def turn_green_on(self):
        self.green.set_trait(name='value', value=self.green_on)

    def turn_green_off(self):
        self.green.set_trait(name='value', value=self.green_off)

    #TIMERS:

    def start_idle_timer(self):
        self.stm.start_timer('idle', 5000)
    def destroy_idle_timer(self):
        self.stm.stop_timer('idle')

    def start_sample_server_response_timer(self):
        self.stm.start_timer('server', 1000)
    def destroy_sample_server_response_timer(self):
        self.stm.stop_timer('server')

    def start_sample_controller_response_timer(self):
        self.stm.start_timer('controller', 1000)
    def destroy_sample_controller_response_timer(self):
        self.stm.stop_timer('controller')

    def start_green_display_timer(self):
        self.stm.start_timer('green_display', 1000)
    def destroy_green_display_timer(self):
        self.stm.stop_timer('green_display')

    def start_red_display_timer(self):
        self.stm.start_timer('red_display', 1000)
    def destroy_red_display_timer(self):
        self.stm.stop_timer('red_display')

    #INFORMATION:

    def info_card_check(self):
        self.info.set_trait(name='value', value="Validating ID.")

    def info_card_add(self):
        self.info.set_trait(name='value', value="ID added to the system.")

    def info_card_remove(self):
        self.info.set_trait(name='value', value="ID removed from the system.")

    def info_server_transition(self):
        self.info.set_trait(name='value', value="Updating Server.")

    def info_waiting_transition(self):
        self.info.set_trait(name='value', value="Waiting...")

    def info_green_state(self):
        self.info.set_trait(name='value', value="Thank you.")

    def info_red_state(self):
        self.info.set_trait(name='value', value="There was a mistake, please try again.")

    #def info_update_label_array(self):
    #self.list_of_ids = [str(i) for i in self.id_array]
    #widgets.HBox(self.list_of_ids)

    def info_update_bar(self):
        self.room_state.set_trait(name='value', value=len(self.id_array))

    #STATE FUNCTIONALITY

    def state_waiting(self):
        self.turn_red_off()
        self.turn_green_off()
        self.start_idle_timer()
        self.info_waiting_transition()

    def state_get_card(self):
        self.turn_red_off()
        self.turn_green_off()
        self.start_sample_controller_response_timer()
        self.info_card_check()

    def state_green(self):
        self.turn_red_off()
        self.turn_green_on()
        self.info_green_state()
        self.start_green_display_timer()

    def state_red(self):
        self.turn_red_on()
        self.turn_green_off()
        self.info_red_state()
        self.start_red_display_timer()

    def state_update_server(self):
        self.turn_red_on()
        self.turn_green_off()
        self.start_sample_server_response_timer()
        self.info_server_transition()
        # here we will send the updated info to the server


    def function_new_card(self, id):  # this represents what we do if we detect a new card
        self.info_card_add()
        print("ID added: ", id, self.id_array)
        self.info_update_bar()  #self.info_update_label_array(id)

    def function_old_card(self, id):  # this represents what we do if we detect a card already in our system
        self.info_card_remove()
        print("ID removed: ", id, self.id_array)
        self.info_update_bar()  #self.info_update_label_array()


    # The following is just a random simulation of the predicted behavior
    def get_random_card(self):
        card_id = random.randint(1, 10)
        if card_id == 1:  # we say a card fails in 10% of the cases
            return 'red'
        else:
            if self.id_array.count(card_id) == 0:
                self.id_array.append(card_id)
                self.function_new_card(card_id)
                return 'green'
            else:
                self.id_array.remove(card_id)
                self.function_old_card(card_id)
                return 'green'


In [3]:
studyHall = StudyHall()

#TRANSITIONS

initiate = {
    'source':'initial',
    'target':'waiting'
}

new_card = {
    'trigger':'newCard',
    'source':'waiting',
    'target':'get_card',
    'effect':'destroy_idle_timer'
    # here we will send the card ID to the controller, and wait for an answer ('effect':'send_ID_to_controller')
}

random_card = { # this is just a simulation of the real behavior
    'trigger':'controller',
    'source':'get_card',
    'function':studyHall.get_random_card
}

green_done = {
    'trigger':'green_display',
    'source':'green',
    'target':'waiting'
}
red_done = {
    'trigger':'red_display',
    'source':'red',
    'target':'waiting'
}



sensor_idle = {
    'trigger':'idle',
    'source':'waiting',
    'target':'update_server'
}
server_done = {
    'trigger':'server',
    'source':'update_server',
    'target':'waiting'
}


#STATES

waiting = {
    'name': 'waiting',
    'entry': 'state_waiting'
}

get_card = {
    'name': 'get_card',
    'entry': 'state_get_card'
    # here we will stay until we receive an answer from the controller, then we change states accordingly
}

green = {
    'name': 'green',
    'entry': 'state_green'
}

red = {
    'name': 'red',
    'entry': 'state_red'
}

update_server = {
    'name': 'update_server',
    'entry': 'state_update_server'
}


stm_studyHall = Machine(name='studyHall', transitions=[initiate, sensor_idle, server_done, new_card, random_card, green_done, red_done], obj=studyHall, states=[waiting, update_server, get_card, green, red])
studyHall.stm = stm_studyHall

driver = Driver()
driver.add_machine(stm_studyHall)
driver.start()

FileNotFoundError: [Errno 2] No such file or directory: 'images/red_on.png'